Import req

In [1]:
import os
import geopandas
import leafmap.kepler as leafmap
from dotenv import load_dotenv,find_dotenv

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
load_dotenv(find_dotenv())

True

Add two layers and reproject it (meters not degrees)

In [3]:
con = leafmap.connect_postgis(database=os.getenv('DATABASE'), host=os.getenv('DBHOST'), user=os.getenv('DBUSER'), password=os.getenv('DBPASSWORD'))
# sql_Q = "SELECT * FROM public.utility"
sql_Q = "SELECT u.id, c.cat_name, u.sub_type, u.lat, u.lon, u.timestamp, u.geometry, u.effect, u.description, u.solved, u.solved_time FROM public.utility u INNER JOIN public.categories c ON c.id=u.type where c.cat_name='Water'"
fires = leafmap.read_postgis(sql_Q, con, geom_col='geometry').to_crs("EPSG:3857")

In [4]:
# fires = geopandas.read_file("../Data/Facilities/DemoCairo.gpkg", layer='fire').to_crs("EPSG:3857") #Point
# facilities = geopandas.read_file("../Data/Facilities/DemoCairo.gpkg", layer='facilities').to_crs("EPSG:3857") #Point
facilities = geopandas.read_file("../Data/Facilities/Incidents.gpkg", layer='Incidents').to_crs("EPSG:3857") #Point
# admin_poly = geopandas.read_file("../Data/Facilities/DemoCairo.gpkg", layer='NewCairoPolyDemo').to_crs("EPSG:3857") #Polygon
admin_poly = geopandas.read_file("../Data/Facilities/Admin3Poly.gpkg", layer='All-Admin-Area-Egypt').to_crs("EPSG:3857")  # Polygon
fires.head(50)
facilities.head(50)

,osm_id,code,fclass,name,email,phone,geometry
0,31272168,5206,motorway_junction,None,org2@youssefharby.com,01040600508,POINT (3492086.393 3517769.836)
1,31321879,5204,crossing,None,org2@youssefharby.com,01040600508,POINT (3475200.228 3654917.638)
2,31321880,5204,crossing,None,org2@youssefharby.com,01040600508,POINT (3475213.575 3654925.691)
3,32521406,5206,motorway_junction,None,org2@youssefharby.com,01040600508,POINT (3492440.188 3517950.768)
4,32537487,5204,crossing,None,org2@youssefharby.com,01040600508,POINT (3485775.668 3515211.387)
5,32537745,5201,traffic_signals,None,org2@youssefharby.com,01040600508,POINT (3485720.476 3515252.378)
6,32538015,5201,traffic_signals,ميدان الجامع,org2@youssefharby.com,01040600508,POINT (3487421.293 3515956.264)
7,33471021,5250,fuel,شل,org2@youssefharby.com,01040600508,POINT (3493226.237 3513679.711)
8,33471101,5206,motorway_junction,None,org2@youssefharby.com,01040600508,POINT (3518426.699 3517620.396)
9,34712107,5250,fuel,Misr Petrol,org2@youssefharby.com,01040600508,POINT (3465140.797 3501848.554)


In [5]:
fac_And_Admin = geopandas.sjoin(facilities,admin_poly[['adm3_ar', 'geometry']], how='left')
fac_And_Admin.rename_geometry('fac_geo', inplace=True)
# fac_And_Admin_for_fire = fac_And_Admin.loc[fac_And_Admin['type']=='Fire']
# fac_And_Admin_for_fire = fac_And_Admin.loc[fac_And_Admin['adm3_ar']=='Industrial Region East']
fac_And_Admin_for_fire = fac_And_Admin
toto = fac_And_Admin_for_fire.to_crs("EPSG:4326")
toto['fac_lat'] = toto['fac_geo'].y
toto['fac_lon'] = toto['fac_geo'].x
geometry = geopandas.points_from_xy(toto.fac_lon, toto.fac_lat)
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in toto.fac_geo]
# print(geo_df_list)

In [6]:
problem_And_Admin = geopandas.sjoin(fires,admin_poly[['adm3_ar', 'geometry']], how='left')
problem_And_Admin.head(10)

,id,cat_name,sub_type,lat,lon,timestamp,geometry,effect,description,solved,solved_time,index_right,adm3_ar
0,1,Water,Leakage,31.206619,29.9627,2022-03-21 02:26:09.395475,POINT (3335432.507 3659611.726),2,NOOO,False,None,3740,عزبة النزهه


In [7]:
toto = fac_And_Admin_for_fire.to_crs("EPSG:4326")
toto['fac_lat'] = toto['fac_geo'].y
toto['fac_lon'] = toto['fac_geo'].x
toto.head(50)

,osm_id,code,fclass,name,email,phone,fac_geo,index_right,adm3_ar,fac_lat,fac_lon
0,31272168,5206,motorway_junction,None,org2@youssefharby.com,01040600508,POINT (31.36995 30.11056),5007,مطار القاهرة,30.110565,31.369946
1,31321879,5204,crossing,None,org2@youssefharby.com,01040600508,POINT (31.21825 31.17055),4851,مدينة بيلا,31.170546,31.218255
2,31321880,5204,crossing,None,org2@youssefharby.com,01040600508,POINT (31.21837 31.17061),4851,مدينة بيلا,31.170608,31.218375
3,32521406,5206,motorway_junction,None,org2@youssefharby.com,01040600508,POINT (31.37312 30.11197),3517,شيراتون المطار,30.111971,31.373124
4,32537487,5204,crossing,None,org2@youssefharby.com,01040600508,POINT (31.31326 30.09068),5278,منشية البكري,30.090681,31.313256
5,32537745,5201,traffic_signals,None,org2@youssefharby.com,01040600508,POINT (31.31276 30.09100),5278,منشية البكري,30.091000,31.312760
6,32538015,5201,traffic_signals,ميدان الجامع,org2@youssefharby.com,01040600508,POINT (31.32804 30.09647),437,البستان,30.096470,31.328038
7,33471021,5250,fuel,شل,org2@youssefharby.com,01040600508,POINT (31.38019 30.07878),3741,عزبة الهجانة,30.078775,31.380185
8,33471101,5206,motorway_junction,None,org2@youssefharby.com,01040600508,POINT (31.60656 30.10940),1718,الفردوس و الكوثر,30.109403,31.606565
9,34712107,5250,fuel,Misr Petrol,org2@youssefharby.com,01040600508,POINT (31.12789 29.98676),5566,نزلة السمان,29.986764,31.127889


In [8]:
# problem_And_Admin.head(50)

In [9]:
final_result = problem_And_Admin.merge(toto[['fac_lat','fac_lon','adm3_ar']], on='adm3_ar',how='left').to_crs("EPSG:4326")
final_result['lat'] = final_result['geometry'].y
final_result['lon'] = final_result['geometry'].x
final_result.head(50)
# final_result.to_json()

,id,cat_name,sub_type,lat,lon,timestamp,geometry,effect,description,solved,solved_time,index_right,adm3_ar,fac_lat,fac_lon
0,1,Water,Leakage,31.206619,29.9627,2022-03-21 02:26:09.395475,POINT (29.96270 31.20662),2,NOOO,False,None,3740,عزبة النزهه,31.212863,29.973270
1,1,Water,Leakage,31.206619,29.9627,2022-03-21 02:26:09.395475,POINT (29.96270 31.20662),2,NOOO,False,None,3740,عزبة النزهه,31.212732,29.973001
2,1,Water,Leakage,31.206619,29.9627,2022-03-21 02:26:09.395475,POINT (29.96270 31.20662),2,NOOO,False,None,3740,عزبة النزهه,31.215298,29.967855
3,1,Water,Leakage,31.206619,29.9627,2022-03-21 02:26:09.395475,POINT (29.96270 31.20662),2,NOOO,False,None,3740,عزبة النزهه,31.216924,29.965808


Save To GPKG

In [10]:
# final_result.to_file('../Data/Facilities/DemoCairoResults.gpkg', driver='GPKG', layer='Result2')


https://kepler.gl/demo/map?mapUrl=https://dl.dropboxusercontent.com/s/1tk4rj86u5f7pj9/keplergl_mxg8ozq.json

In [11]:
m = leafmap.Map()

In [14]:
config = "../CONFIGs/SpatialJoin/config-sj-try6-label.json"

In [15]:
m.add_gdf(final_result, layer_name="Final Result")
m.add_gdf(admin_poly, layer_name="Admin Area", config=config)
m

Map(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['Final Result'], 'id': '65tlkme8k…

In [ ]:
# m.save_config("../CONFIGs/config-sj-try7-label.json")